## Downloading Very Large CSV File

### CORD-19 Dataset

In [1]:
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-02-07.tar.gz

--2022-02-15 09:50:29--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-02-07.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.146.106
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.146.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15605406166 (15G) [binary/octet-stream]
Saving to: ‘cord-19_2022-02-07.tar.gz’

cord-19_2022-02-07. 100%[===================>]  14.53G  33.6MB/s    in 8m 8s   

2022-02-15 09:58:38 (30.5 MB/s) - ‘cord-19_2022-02-07.tar.gz’ saved [15605406166/15605406166]



### Unzip CSV file

In [21]:
!tar -xvzf "cord-19_2022-02-07.tar.gz"

2022-02-07/changelog
2022-02-07/cord_19_embeddings.tar.gz
2022-02-07/document_parses.tar.gz
2022-02-07/metadata.csv


In [25]:
!tar -xvzf "/content/2022-02-07/cord_19_embeddings.tar.gz"

./cord_19_embeddings_2022-02-07.csv


## Check My PC Spec

In [4]:
!grep MemTotal /proc/meminfo

MemTotal:       13302920 kB


In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        1.3G        3.5G        1.5M        7.8G         11G
Swap:            0B          0B          0B


In [12]:
!ls -lh

total 28G
drwxr-xr-x 2 root root 4.0K Feb 15 10:06 2022-02-07
-rw-r--r-- 1 root root  15G Feb 11 13:52 cord-19_2022-02-07.tar.gz
-rw-r--r-- 1 root root  13G Feb 11 12:02 cord_19_embeddings_2022-02-07.csv
drwxr-xr-x 1 root root 4.0K Feb  1 14:32 sample_data


## Installing Memit

In [1]:
!pip install memory_profiler

In [2]:
%load_ext memory_profiler

## Method 1: Loading Entire Dataset into RAM

In [5]:
import pandas as pd
import time

In [6]:
%%memit 

startTime = time.time()

BIGdf = pd.read_csv("/content/cord_19_embeddings_2022-02-07.csv")

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")

It takes  264.7972409725189  seconds
peak memory: 11127.07 MiB, increment: 10994.32 MiB


In [7]:
%memit

BIGdf["-2.939983606338501"].sum()

peak memory: 11074.74 MiB, increment: -52.10 MiB


-1505758.9940468734

In [8]:
%memit

BIGdf[BIGdf["-2.939983606338501"] > 5.5]

peak memory: 11069.23 MiB, increment: -7.07 MiB


,ug7v899j,-2.939983606338501,-6.312200546264648,-1.0459030866622925,5.164162635803223,-0.32564637064933777,-2.507413387298584,1.735608696937561,1.9363566637039185,0.622501015663147,1.5613162517547607,2.7476913928985596,-0.693097710609436,-0.371040940284729,-5.885725021362305,-1.097484827041626,-4.654815673828125,-2.3164637088775635,2.8749594688415527,3.8757808208465576,-0.29598113894462585,1.7702548503875732,4.400593280792236,-0.40593475103378296,-1.7162439823150635,-0.47666341066360474,-2.570713758468628,4.599417686462402,0.16352784633636475,-1.8347313404083252,2.5578644275665283,-3.7916784286499023,-6.2248969078063965,-0.353383868932724,-1.083785057067871,-0.10440695285797119,-2.098464012145996,-1.631538987159729,-0.05292006582021713,-4.062861442565918,...,-4.448026657104492,0.6586090326309204,2.146946668624878,2.1082308292388916,1.6172025203704834,1.3320376873016357,2.582035541534424,0.38238826394081116,2.0623371601104736,-1.6251556873321533,-3.5131406784057617,5.951539993286133,-1.8060410022735596,4.870253086090088,2.026643991470337,-1.3290166854858398,3.488175868988037,-3.4803996086120605,-3.017104148864746,1.3051470518112183,4.799997329711914,2.0917749404907227,3.655892848968506,7.50132417678833,1.7296870946884155,-2.760751247406006,0.6788414716720581,9.059469223022461,-6.121626377105713,-0.28331542015075684,-0.7419174313545227,3.3947110176086426,3.163691282272339,1.3728902339935303,-4.107113361358643,-3.8174145221710205,1.8976187705993652,5.811529159545898,-2.9323840141296387,0.30520474910736084
297,l1khrk52,6.705245,-4.412942,1.679989,6.228924,2.269611,-2.327944,-3.228223,-0.847615,0.900813,-2.753817,0.171072,-6.229174,-2.996866,-0.864276,-2.096279,0.125930,-1.796674,-0.887222,-0.588666,3.047506,-1.853603,-0.526515,-2.758754,-0.945638,3.221270,-0.682561,0.537505,-1.530583,-0.113680,-1.608841,-1.253872,2.044073,1.378250,-6.907540,7.933252,-3.039894,-2.070267,5.301340,2.411879,...,1.268878,-0.207003,3.377431,2.002836,-0.630085,1.910574,-3.182068,4.390864,1.092352,-0.326957,3.553081,1.631736,0.538246,2.293353,2.512174,-5.013056,4.541186,1.853157,-3.978210,3.830422,1.378477,-7.648742,3.929116,1.334061,1.696364,-6.166232,1.574295,6.531921,3.972760,1.597135,2.359899,1.117025,0.373170,2.204131,0.858064,-6.403036,-1.490938,-1.567014,-1.035617,-0.874245
1344,5s10b9zy,5.793284,-7.490951,-0.138165,7.755403,-0.836501,3.265542,-2.141375,0.196434,-0.525601,0.747793,0.629116,-3.527104,-1.300196,-2.648640,-1.628619,-5.264542,-1.790191,2.735249,4.377747,-4.954486,2.924777,0.414951,-3.119133,-6.798826,0.136178,-2.189961,-2.009782,1.122020,-0.590301,1.514654,-2.397801,-5.230255,-4.409037,3.274498,0.819697,-2.344497,1.405164,2.775282,-0.475403,...,0.812003,-1.100130,3.299105,1.838783,0.055178,4.622440,1.878831,-6.229831,5.831432,-0.524835,0.480841,1.153823,0.925086,-0.426815,-0.789165,2.508557,1.562358,4.893148,-2.331898,3.854482,-0.254871,4.534487,4.069223,5.520227,2.678321,-3.308801,0.318559,7.687768,-3.710832,-1.745678,1.258802,3.301752,1.335237,2.983571,0.736282,0.556859,-0.415278,4.139990,0.211016,7.711371
2504,dr1c9ttb,6.111768,-6.923980,1.878374,1.104449,1.321022,-2.661004,-3.420686,-6.324624,-0.773544,-1.911280,4.741815,-2.717345,3.122451,-2.077291,-0.369376,0.745390,-2.007006,5.858207,1.513402,-1.987694,1.789805,1.648632,-2.420773,-2.422765,1.339028,0.363227,-5.360324,1.001470,-2.348048,3.122884,-5.198231,-0.008631,3.830672,0.747222,1.296304,-3.360209,-2.636691,5.363902,-3.661054,...,-1.542641,3.720917,5.279918,-4.232904,0.036626,2.095180,-1.310886,-8.637354,3.080811,-1.380744,3.948786,4.890907,1.478894,-0.553277,-2.551219,-2.988999,1.025052,-0.627876,-3.385152,2.286301,-3.993875,-0.140712,2.129278,7.303324,3.892272,-2.427576,0.231819,7.612614,1.418565,1.216913,-0.559581,3.561411,-0.134380,-3.659540,-1.348839,-0.391091,4.077968,6.442573,-6.471324,2.405981
2529,q6zfd3fs,5.614570,-5.305712,1.740511,8.630479,2.135732,-0.784876,-3.244976,1.286346,-2.188538,-1.280988,4.553161,-1.619282,-3.518452,-4.585254,-0.209429,-2.543436,-1.488222

In [9]:
!free -h 

              total        used        free      shared  buff/cache   available
Mem:            12G         11G        279M        1.4M        760M        5.9G
Swap:            0B          0B          0B


In [10]:
!free --gibi

              total        used        free      shared  buff/cache   available
Mem:             12          11           0           0           0           5
Swap:             0           0           0


## Method 2: Pandas Using Chunksize

In [3]:
import pandas as pd
import time

In [26]:
%%memit -r1

startTime = time.time()

#read data in chunks of 1 million rows at a time
df_chunk = pd.read_csv('/content/cord_19_embeddings_2022-02-07.csv',chunksize=10_000, low_memory=True, iterator=True)

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")


It takes  0.01427006721496582  seconds
It takes  0.011810064315795898  seconds
It takes  0.012421846389770508  seconds
peak memory: 183.01 MiB, increment: 0.12 MiB


In [27]:
%%memit 

startTime = time.time()

chunk_sum = 0
chunk_filter = []
for chunk in df_chunk:  
    # perform sum 
    chunk_sum = chunk_sum + chunk["-2.939983606338501"].sum()

    # perform filtering
    temp_df = chunk[chunk["-2.939983606338501"] > 5.5]
    chunk_filter.append(temp_df)

result_df = pd.concat(chunk_filter)

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")

result_df

It takes  223.22706484794617  seconds
peak memory: 432.77 MiB, increment: 249.76 MiB


In [28]:
result_df

,ug7v899j,-2.939983606338501,-6.312200546264648,-1.0459030866622925,5.164162635803223,-0.32564637064933777,-2.507413387298584,1.735608696937561,1.9363566637039185,0.622501015663147,1.5613162517547607,2.7476913928985596,-0.693097710609436,-0.371040940284729,-5.885725021362305,-1.097484827041626,-4.654815673828125,-2.3164637088775635,2.8749594688415527,3.8757808208465576,-0.29598113894462585,1.7702548503875732,4.400593280792236,-0.40593475103378296,-1.7162439823150635,-0.47666341066360474,-2.570713758468628,4.599417686462402,0.16352784633636475,-1.8347313404083252,2.5578644275665283,-3.7916784286499023,-6.2248969078063965,-0.353383868932724,-1.083785057067871,-0.10440695285797119,-2.098464012145996,-1.631538987159729,-0.05292006582021713,-4.062861442565918,...,-4.448026657104492,0.6586090326309204,2.146946668624878,2.1082308292388916,1.6172025203704834,1.3320376873016357,2.582035541534424,0.38238826394081116,2.0623371601104736,-1.6251556873321533,-3.5131406784057617,5.951539993286133,-1.8060410022735596,4.870253086090088,2.026643991470337,-1.3290166854858398,3.488175868988037,-3.4803996086120605,-3.017104148864746,1.3051470518112183,4.799997329711914,2.0917749404907227,3.655892848968506,7.50132417678833,1.7296870946884155,-2.760751247406006,0.6788414716720581,9.059469223022461,-6.121626377105713,-0.28331542015075684,-0.7419174313545227,3.3947110176086426,3.163691282272339,1.3728902339935303,-4.107113361358643,-3.8174145221710205,1.8976187705993652,5.811529159545898,-2.9323840141296387,0.30520474910736084
297,l1khrk52,6.705245,-4.412942,1.679989,6.228924,2.269611,-2.327944,-3.228223,-0.847615,0.900813,-2.753817,0.171072,-6.229174,-2.996866,-0.864276,-2.096279,0.125930,-1.796674,-0.887222,-0.588666,3.047506,-1.853603,-0.526515,-2.758754,-0.945638,3.221270,-0.682561,0.537505,-1.530583,-0.113680,-1.608841,-1.253872,2.044073,1.378250,-6.907540,7.933252,-3.039894,-2.070267,5.301340,2.411879,...,1.268878,-0.207003,3.377431,2.002836,-0.630085,1.910574,-3.182068,4.390864,1.092352,-0.326957,3.553081,1.631736,0.538246,2.293353,2.512174,-5.013056,4.541186,1.853157,-3.978210,3.830422,1.378477,-7.648742,3.929116,1.334061,1.696364,-6.166232,1.574295,6.531921,3.972760,1.597135,2.359899,1.117025,0.373170,2.204131,0.858064,-6.403036,-1.490938,-1.567014,-1.035617,-0.874245
1344,5s10b9zy,5.793284,-7.490951,-0.138165,7.755403,-0.836501,3.265542,-2.141375,0.196434,-0.525601,0.747793,0.629116,-3.527104,-1.300196,-2.648640,-1.628619,-5.264542,-1.790191,2.735249,4.377747,-4.954486,2.924777,0.414951,-3.119133,-6.798826,0.136178,-2.189961,-2.009782,1.122020,-0.590301,1.514654,-2.397801,-5.230255,-4.409037,3.274498,0.819697,-2.344497,1.405164,2.775282,-0.475403,...,0.812003,-1.100130,3.299105,1.838783,0.055178,4.622440,1.878831,-6.229831,5.831432,-0.524835,0.480841,1.153823,0.925086,-0.426815,-0.789165,2.508557,1.562358,4.893148,-2.331898,3.854482,-0.254871,4.534487,4.069223,5.520227,2.678321,-3.308801,0.318559,7.687768,-3.710832,-1.745678,1.258802,3.301752,1.335237,2.983571,0.736282,0.556859,-0.415278,4.139990,0.211016,7.711371
2504,dr1c9ttb,6.111768,-6.923980,1.878374,1.104449,1.321022,-2.661004,-3.420686,-6.324624,-0.773544,-1.911280,4.741815,-2.717345,3.122451,-2.077291,-0.369376,0.745390,-2.007006,5.858207,1.513402,-1.987694,1.789805,1.648632,-2.420773,-2.422765,1.339028,0.363227,-5.360324,1.001470,-2.348048,3.122884,-5.198231,-0.008631,3.830672,0.747222,1.296304,-3.360209,-2.636691,5.363902,-3.661054,...,-1.542641,3.720917,5.279918,-4.232904,0.036626,2.095180,-1.310886,-8.637354,3.080811,-1.380744,3.948786,4.890907,1.478894,-0.553277,-2.551219,-2.988999,1.025052,-0.627876,-3.385152,2.286301,-3.993875,-0.140712,2.129278,7.303324,3.892272,-2.427576,0.231819,7.612614,1.418565,1.216913,-0.559581,3.561411,-0.134380,-3.659540,-1.348839,-0.391091,4.077968,6.442573,-6.471324,2.405981
2529,q6zfd3fs,5.614570,-5.305712,1.740511,8.630479,2.135732,-0.784876,-3.244976,1.286346,-2.188538,-1.280988,4.553161,-1.619282,-3.518452,-4.585254,-0.209429,-2.543436,-1.488222

In [4]:
chunk_sum

-1505758.9940468734

# Method 3: Using Dask

In [3]:
!python -m pip install "dask[dataframe]" 
!python -m pip install 'fsspec>=0.3.3'

In [4]:
from dask import dataframe as dd
import time

In [5]:
%%memit -r1

startTime = time.time()

dask_df = dd.read_csv('/content/cord_19_embeddings_2022-02-07.csv')

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")

It takes  9.110302209854126  seconds
peak memory: 173.91 MiB, increment: 31.15 MiB


In [6]:
%%memit -r1

start = time.time()

dask_df["-2.939983606338501"].sum().compute()

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")

It takes  253.58624792099  seconds
peak memory: 792.21 MiB, increment: 618.10 MiB


In [7]:
%%memit -r1

start = time.time()

dask_df[dask_df["-2.939983606338501"] > 5.5].compute()

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")

It takes  517.8697140216827  seconds
peak memory: 843.69 MiB, increment: 79.89 MiB


In [9]:
%%memit -r1
import dask

start = time.time()

result_sum, result_df = dask.compute(
                                     dask_df["-2.939983606338501"].sum(),
                                     dask_df[dask_df["-2.939983606338501"] > 5.5]
                                     )

finishTime = time.time() - startTime
print("It takes ", finishTime, " seconds")

It takes  769.2557890415192  seconds
peak memory: 846.09 MiB, increment: 15.00 MiB


In [11]:
result_sum

-1505758.9940468734

In [10]:
result_df

,ug7v899j,-2.939983606338501,-6.312200546264648,-1.0459030866622925,5.164162635803223,-0.32564637064933777,-2.507413387298584,1.735608696937561,1.9363566637039185,0.622501015663147,1.5613162517547607,2.7476913928985596,-0.693097710609436,-0.371040940284729,-5.885725021362305,-1.097484827041626,-4.654815673828125,-2.3164637088775635,2.8749594688415527,3.8757808208465576,-0.29598113894462585,1.7702548503875732,4.400593280792236,-0.40593475103378296,-1.7162439823150635,-0.47666341066360474,-2.570713758468628,4.599417686462402,0.16352784633636475,-1.8347313404083252,2.5578644275665283,-3.7916784286499023,-6.2248969078063965,-0.353383868932724,-1.083785057067871,-0.10440695285797119,-2.098464012145996,-1.631538987159729,-0.05292006582021713,-4.062861442565918,...,-4.448026657104492,0.6586090326309204,2.146946668624878,2.1082308292388916,1.6172025203704834,1.3320376873016357,2.582035541534424,0.38238826394081116,2.0623371601104736,-1.6251556873321533,-3.5131406784057617,5.951539993286133,-1.8060410022735596,4.870253086090088,2.026643991470337,-1.3290166854858398,3.488175868988037,-3.4803996086120605,-3.017104148864746,1.3051470518112183,4.799997329711914,2.0917749404907227,3.655892848968506,7.50132417678833,1.7296870946884155,-2.760751247406006,0.6788414716720581,9.059469223022461,-6.121626377105713,-0.28331542015075684,-0.7419174313545227,3.3947110176086426,3.163691282272339,1.3728902339935303,-4.107113361358643,-3.8174145221710205,1.8976187705993652,5.811529159545898,-2.9323840141296387,0.30520474910736084
297,l1khrk52,6.705245,-4.412942,1.679989,6.228924,2.269611,-2.327944,-3.228223,-0.847615,0.900813,-2.753817,0.171072,-6.229174,-2.996866,-0.864276,-2.096279,0.125930,-1.796674,-0.887222,-0.588666,3.047506,-1.853603,-0.526515,-2.758754,-0.945638,3.221270,-0.682561,0.537505,-1.530583,-0.113680,-1.608841,-1.253872,2.044073,1.378250,-6.907540,7.933252,-3.039894,-2.070267,5.301340,2.411879,...,1.268878,-0.207003,3.377431,2.002836,-0.630085,1.910574,-3.182068,4.390864,1.092352,-0.326957,3.553081,1.631736,0.538246,2.293353,2.512174,-5.013056,4.541186,1.853157,-3.978210,3.830422,1.378477,-7.648742,3.929116,1.334061,1.696364,-6.166232,1.574295,6.531921,3.972760,1.597135,2.359899,1.117025,0.373170,2.204131,0.858064,-6.403036,-1.490938,-1.567014,-1.035617,-0.874245
1344,5s10b9zy,5.793284,-7.490951,-0.138165,7.755403,-0.836501,3.265542,-2.141375,0.196434,-0.525601,0.747793,0.629116,-3.527104,-1.300196,-2.648640,-1.628619,-5.264542,-1.790191,2.735249,4.377747,-4.954486,2.924777,0.414951,-3.119133,-6.798826,0.136178,-2.189961,-2.009782,1.122020,-0.590301,1.514654,-2.397801,-5.230255,-4.409037,3.274498,0.819697,-2.344497,1.405164,2.775282,-0.475403,...,0.812003,-1.100130,3.299105,1.838783,0.055178,4.622440,1.878831,-6.229831,5.831432,-0.524835,0.480841,1.153823,0.925086,-0.426815,-0.789165,2.508557,1.562358,4.893148,-2.331898,3.854482,-0.254871,4.534487,4.069223,5.520227,2.678321,-3.308801,0.318559,7.687768,-3.710832,-1.745678,1.258802,3.301752,1.335237,2.983571,0.736282,0.556859,-0.415278,4.139990,0.211016,7.711371
2504,dr1c9ttb,6.111768,-6.923980,1.878374,1.104449,1.321022,-2.661004,-3.420686,-6.324624,-0.773544,-1.911280,4.741815,-2.717345,3.122451,-2.077291,-0.369376,0.745390,-2.007006,5.858207,1.513402,-1.987694,1.789805,1.648632,-2.420773,-2.422765,1.339028,0.363227,-5.360324,1.001470,-2.348048,3.122884,-5.198231,-0.008631,3.830672,0.747222,1.296304,-3.360209,-2.636691,5.363902,-3.661054,...,-1.542641,3.720917,5.279918,-4.232904,0.036626,2.095180,-1.310886,-8.637354,3.080811,-1.380744,3.948786,4.890907,1.478894,-0.553277,-2.551219,-2.988999,1.025052,-0.627876,-3.385152,2.286301,-3.993875,-0.140712,2.129278,7.303324,3.892272,-2.427576,0.231819,7.612614,1.418565,1.216913,-0.559581,3.561411,-0.134380,-3.659540,-1.348839,-0.391091,4.077968,6.442573,-6.471324,2.405981
2529,q6zfd3fs,5.614570,-5.305712,1.740511,8.630479,2.135732,-0.784876,-3.244976,1.286346,-2.188538,-1.280988,4.553161,-1.619282,-3.518452,-4.585254,-0.209429,-2.543436,-1.488222